## **深層学習　画像データセットで分類を行う**

# **STEP0：事前準備**

**コード2-1 Google Driveのマウント**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

**コード2-2 zipファイルを解凍する**

In [ ]:
!unzip '/content/drive/MyDrive/images_TMDU_2023.zip'

**コード2-3 ライブラリ・モジュール・クラスをインポート**

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import set_random_seed

**コード2-4 画像の読み込みと図示**

In [ ]:
test = load_img('/content/images/test.jpg',
                 color_mode='grayscale',
                 target_size=(10,10))
plt.imshow(test, 'gray')
plt.show()
print(test)

**コード2-5 画像データを配列データに変換**

In [ ]:
test_array = img_to_array(test)
print(test_array)
print(test_array.shape)

# **STEP1：データの用意と前処理（説明をしないので、コード2-1-1〜コード2-1-17まではそのまま実行してください）**

**コード2-1-1 ファイル名のリストを作成**

In [6]:
list_healthy = [i for i in os.listdir('/content/images/COVID-NORMAL/healthy') if not i.startswith('.')]
list_covid19 = [i for i in os.listdir('/content/images/COVID-NORMAL/covid19') if not i.startswith('.')]

**コード2-1-7 画像ファイル数の集計**

In [ ]:
num_healthy = len(list_healthy)
num_covid19 = len(list_covid19)
num_all = num_healthy + num_covid19
print(num_all)

**コード2-1-8 画像データ格納用numpy配列の作成**

In [ ]:
images_temp = np.zeros((num_all,64,64,1), dtype = float)
labels_temp = np.zeros((num_all, 1), dtype = int)
print(images_temp.shape)
print(labels_temp.shape)

**コード2-1-10 画像データの読み込み「list_healthy」**

In [9]:
path = '/content/images/COVID-NORMAL'
for i in range(num_healthy):
  file = f'{path}/healthy/{list_healthy[i]}'
  file_img = load_img(file, color_mode='grayscale', target_size = (64,64), interpolation='lanczos')
  images_temp[i]=img_to_array(file_img)/255

**コード2-1-13 画像データの読み込み「list_covid19」**

In [10]:
for i in range(num_covid19):
  file = f'{path}/covid19/{list_covid19[i]}'
  file_img = load_img(file, color_mode='grayscale', target_size = (64,64), interpolation='lanczos')
  images_temp[i + num_healthy] = img_to_array(file_img)/255
  labels_temp[i + num_healthy] = 1

**コード2-1-15~16 0から231までの数字の配列をシャッフルして作成**

In [ ]:
np.random.seed(0)
num_list = np.arange(num_all)
np.random.shuffle(num_list)
print(num_list)

**コード2-1-17 images_tempとlabels_tempの配列をシャッフルする**

In [12]:
x_train = images_temp[num_list]
y_train = labels_temp[num_list]

# **STEP2：モデルの選択**

**コード2-6 ランダムシード値の設定**

In [13]:
set_random_seed(0)
tf.config.experimental.enable_op_determinism()

**コード2-7 モデルの設計**

In [ ]:
model = Sequential()
model.add(Flatten(input_shape = (64,64,1)))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
model.summary()

# **STEP3：データを入れて学習**

**コード2-8 データを入れて学習させる**

In [ ]:
result = model.fit(x_train, y_train,
                   batch_size = 32,
                   epochs = 100,
                   validation_split = 0.2)

# **STEP4：モデルの評価**

**コード2-9 正解率の図示**

In [ ]:
plt.plot(result.history['accuracy'], label = 'accuracy')
plt.plot(result.history['val_accuracy'], label = 'val_accuracy')
plt.legend()
plt.show()

**コード2-10 損失の図示**

In [ ]:
plt.plot(result.history['loss'], label = 'loss')
plt.plot(result.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

# **過学習を予測するため、別のモデルを作成**

# **STEP2：学習モデルの選択(model_d)**

**コード2-11 ドロップアウトを入れたモデルの設計**

In [ ]:
model_d = Sequential()
model_d.add(Flatten(input_shape = (64,64,1)))
model_d.add(Dense(512, activation = 'relu'))
model_d.add(Dropout(0.5))
model_d.add(Dense(256, activation = 'relu'))
model_d.add(Dropout(0.5))
model_d.add(Dense(128, activation = 'relu'))
model_d.add(Dense(1, activation = 'sigmoid'))
model_d.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
model_d.summary()


# **STEP3：データを入れて学習**

**コード2-12 データを入れて学習させる**

In [ ]:
result_d = model_d.fit(x_train, y_train,
                       batch_size = 32,
                       epochs = 100,
                       validation_split = 0.2)

# **STEP4：モデルの評価**

**コード2-13 ドロップアウトを加えたモデルの学習過程の図示**

In [ ]:
plt.plot(result_d.history['accuracy'], label = 'accuracy')
plt.plot(result_d.history['val_accuracy'], label = 'val_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(result_d.history['loss'], label = 'loss')
plt.plot(result_d.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

# **STEP5：予測**

**コード2-14 別の画像データ(covid19肺炎のX線写真)を読み込む**

In [22]:
img1 = img_to_array(load_img('/content/images/covid.jpg',
                             color_mode = 'grayscale',
                             target_size = (64,64)))/255
check = np.zeros((1,64,64,1))
check[0] = img1

**コード2-15 画像をモデルに当てはめて予測を行う**

In [ ]:
print(model_d.predict(check))

**コード2-16 画像データ（健康な肺のX線写真）を読み込む**

In [24]:
img2 = img_to_array(load_img('/content/images/NORMAL.jpg',
                             color_mode = 'grayscale',
                             target_size = (64,64)))/255
check = np.zeros((1,64,64,1))
check[0] = img2

**コード2-17 画像をモデルに当てはめて予測を行う**

In [ ]:
print(model_d.predict(check))